In [109]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import re

In [30]:
# russian opt1
#tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
#model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# russian opt2
#tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
#model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

# finnish opt1
#tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
#model = AutoModel.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

# finnish opt2
#tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")
#model = AutoModel.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")

# german opt1
#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")

# german opt2
#tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-german-cased")
#model = AutoModel.from_pretrained("google-bert/bert-base-german-cased")

# multilingual
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [13]:
df = pd.read_csv('../data/finnish/axolotl.dev.fi.tsv', sep='\t')
df

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,dev_fi_0,ajainen,"ajaisen,",ajainen_iMPSFeVQEfY,ajallinen,"witzauxen ajaisen, Herra sun päälles heitti",10:18,1700,new
1,dev_fi_1,ajainen,ajainen,ajainen_iMPSFeVQEfY,ajallinen,"ajainen rangaistus nijn aiwan suur, Täs rundel...",0:7,1750,new
2,dev_fi_2,alentaa,"alenna,",alentaa_pJTUhn5-iL4,"laskea alemmaksi, tehdä matalammaksi","Ei tuki suowa alenna, wara wenhettä caada",14:21,1700,new
3,dev_fi_3,alentaa,alettuja,alentaa_pJTUhn5-iL4,"laskea alemmaksi, tehdä matalammaksi",kynnön pitä [tapahtua niin] – – ettei yhtäkän ...,67:75,1750,new
4,dev_fi_4,alentaa,alendaa,alentaa_pJTUhn5-iL4,"laskea alemmaksi, tehdä matalammaksi",on myös hyödyllinen – – maata Tarhan sisä puol...,64:71,1750,new
...,...,...,...,...,...,...,...,...,...
6549,dev_fi_6549,mieli-suosio,mieli suosjost,mieli-suosio_hpsE0n4lHYs,"suostumus, hyväksyntä; suopeus, suosio; vapaae...","Tytär täsä dandzais taitavast, Herodexen mieli...",41:55,1600,old
6550,dev_fi_6550,mieli-suosio,miel suosjo,mieli-suosio_hpsE0n4lHYs,"suostumus, hyväksyntä; suopeus, suosio; vapaae...","Herrall cunnja corkjudhes, Maasa raoha ihmises...",58:69,1600,old
6551,dev_fi_6551,mieli-suosio,mielisuosio,mieli-suosio_hpsE0n4lHYs,"suostumus, hyväksyntä; suopeus, suosio; vapaae...",P. Hengen erinomainen – – hywä tahto ia mielis...,40:51,1600,old
6552,dev_fi_6552,mieli-suosio,Mielisuosion,mieli-suosio_hpsE0n4lHYs,"suostumus, hyväksyntä; suopeus, suosio; vapaae...",Minä N. N. lupan – – että minä ilman catzomist...,65:77,1650,old


In [113]:
def find_sub_list(sl,l): # not used because some examples have no exact coincidence
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind+1,ind+sll # +1 for the [CLS] token
        
def print_nice(input_ids, index):
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    tokens[index] = '\033[91m' + tokens[index] + '\033[0m'
    print(' '.join(tokens))

def find_word_containing_target(sentence, target_word):
    index = sentence.find(target_word)
    if index == -1:
        return None
    start_index = sentence.rfind(" ", 0, index) + 1 if index != 0 else 0
    end_index = sentence.find(" ", index + len(target_word)) if sentence.find(" ", index + len(target_word)) != -1 else len(sentence)
    return sentence[start_index:end_index]

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [117]:
embeddings = []
word = ""
for index, row in df.iterrows():
    if word != "" and word != row['word']:
        print(f"{word}")
    word = row['word']          # target word
    orth = row['orth']          # usage of the target word in the example
    sense_id = row['sense_id']  # sense of the target word in the example
    gloss = row['gloss']        # definition of the target word
    example = row['example']    # usage example of the target word

    # 1. Find the word index in the example
    found_orth = find_word_containing_target(example, orth)
    if found_orth:
        orth = found_orth

    tokens = tokenizer.tokenize(example)
    if tokenizer.tokenize(orth)[0] in tokens:
        target_index = tokens.index(tokenizer.tokenize(orth)[0])+1 # +1 for the [CLS] token
    else:
        # try with the ## prefix
        target_index = tokens.index(f"##{tokenizer.tokenize(orth)[0]}")+1 # +1 for the [CLS] token

    inputs = tokenizer(example, return_tensors="pt")
    #print_nice(inputs['input_ids'][0], target_index)

    with torch.no_grad():
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)

    embeddings.append( outputs.last_hidden_state[0][target_index])

ajainen
alentaa
alho
pruuni
kelvottomuus
ken
kenkään
kernaasti
kerta
huhu
nähtävä
näky
näpeästi
närkästyttää
näyttäminen
perintö-kuningas
perintö-prinssi
oppi-kirja
lähempi
lämmin-tauti
kolkuttaminen
kolminainen
koritus
edes-seisova
edes-toimittaa
edes-vastaaja
edes-vetäminen
ehdollinen
kyllyys
kyllältä
kynä
kypärä
kyyhkynen
kähistä
pois-torjua
pois-viedä
mitta-puu
monialla
osaantua
osallisuus
lahjoittaminen
lahota
kärpänen
kärsiväisyys
kasteen-liitto
kasvanto
leveästi
levätä
liesi
lihallinen
muualla
muun-kaltainen
muutos
kilpi
kirjava
kirkko-kunta
myrsky
märkä-pää
osmundi-rauta
otollinen
otollisuus
prosessi
jousi
julma
jumalaa-pelkääväinen
jumalan-palvelus
maan-jako
maan-kartano
maan-oikeus
maan-ääri
armahtava
armas
arpa
itäinen
iäinen
puhde
puhdistaminen
pitää
pohjaton
kutsuminen
kuuleminen
kuurna
hidas
privilegio
offiseri
erin-omattain
ero-kirja
este
kumina
kunne
kuokkia
kuoleman-hetki
kuoria
kupari
kihlattu
kiinni-lukita
kita
kitistä
kiukkuinen
kivi-raunio
kivistö
kauhistua
kaupan-

In [ ]:
#assert len(embeddings) == len(df), "Embeddings and dataframe have different lengths"
embeddings = torch.stack(embeddings)
embeddings.shape